# Домашнее задание №4. «Рекомендации на основе скрытых факторов»

## “Требуется построить модель рекомендаций на основе скрытых факторов (implicit) на основе dataset’а https://grouplens.org/datasets/hetrec-2011/ (Delicious Bookmarks)”

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

import scipy.sparse as sparse

%matplotlib inline

from implicit.als import AlternatingLeastSquares

In [2]:
data = pd.read_table('user_taggedbookmarks-timestamps.dat', sep='\\n', delimiter='\\t')

C:\Users\Arsenii\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.
C:\Users\Arsenii\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
del data['timestamp']

In [4]:
data.head()

,userID,bookmarkID,tagID
0,8,1,1
1,8,2,1
2,8,7,1
3,8,7,6
4,8,7,7


In [5]:
data = data.groupby(['userID', 'bookmarkID'], as_index=False)['tagID'].count()
data = data.rename(columns={'tagID': 'tags_num'})

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104799 entries, 0 to 104798
Data columns (total 3 columns):
userID        104799 non-null int64
bookmarkID    104799 non-null int64
tags_num      104799 non-null int64
dtypes: int64(3)
memory usage: 3.2 MB


In [7]:
data.head()

,userID,bookmarkID,tags_num
0,8,1,1
1,8,2,1
2,8,7,3
3,8,8,3
4,8,9,2


## Отсутствующих значений нет.

In [8]:
user_id_lookup = dict(zip(range(data['userID'].nunique()), data['userID'].unique()))
item_id_lookup = dict(zip(range(data['bookmarkID'].nunique()), data['bookmarkID'].unique()))
data['userID'] = data['userID'].map({v: k for k, v in user_id_lookup.items()})
data['bookmarkID'] = data['bookmarkID'].map({v: k for k, v in item_id_lookup.items()})

In [9]:
ids = list(np.sort(data['userID'].unique()))
urls = list(np.sort(data['bookmarkID'].unique()))

In [10]:
rows = data['userID']
cols = data['bookmarkID']
tags_num = data['tags_num']

In [11]:
len(tags_num), len(cols), len(rows)

(104799, 104799, 104799)

In [12]:
len(urls), len(ids)

(69223, 1867)

In [13]:
data_sparse = sparse.csr_matrix((tags_num, (cols, rows)), shape=(len(urls), len(ids)))

In [14]:
model = AlternatingLeastSquares(factors=100)
model.fit(data_sparse)

100%|████████████████████████████████████████████████████████████████████████████████| 15.0/15 [00:01<00:00, 11.47it/s]


In [15]:
userid = 100
user_items = data_sparse.transpose()
recommendations = model.recommend(userid, user_items)

In [17]:
recommendations = [(item_id_lookup[rec[0]], rec[1]) for rec in recommendations]

In [18]:
recommendations

[(9132, 0.6274389),
 (4044, 0.6041491),
 (9100, 0.53076446),
 (2618, 0.49297342),
 (9104, 0.49024433),
 (4276, 0.4881544),
 (9092, 0.48351568),
 (4342, 0.48312724),
 (2293, 0.48262194),
 (9108, 0.469109)]

In [19]:
rec_ids = np.array(recommendations)[:, 0]

## Посмотрим что это за закладки (их названия)

In [20]:
titles = pd.read_csv('bookmarks.dat', sep='\\n', delimiter='\\t')
del titles['md5']
del titles['url']
del titles['md5Principal']
del titles['urlPrincipal']

C:\Users\Arsenii\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [21]:
titles.query("id in @rec_ids")

,id,title
1664,2293,The Square Grid - A simple CSS framework for d...
1903,2618,Wordpress Snippets
2831,4044,UI Guidelines for mobile and tablet web app de...
2975,4276,Flipboard for iPad
3023,4342,"Theme Finder, a WPCandy Project"
6514,9092,"Spazio web, database, traffico, servizi cloud ..."
6520,9100,Theme Layouts | ThemeGarden.com
6524,9104,Data Visualization: Modern Approaches | Graphi...
6526,9108,GCALDaemon - Project Home
6542,9132,Tweet Nest


## Посмотрим на похожие items.

In [27]:
itemid = 250
related = model.similar_items(itemid)
related = [(item_id_lookup[rel[0]], rel[1]) for rel in related]

In [28]:
related

[(366, 0.13080156),
 (399, 0.13080156),
 (380, 0.13080156),
 (393, 0.13080156),
 (371, 0.13080156),
 (388, 0.13078338),
 (374, 0.13078336),
 (356, 0.13078336),
 (383, 0.13078335),
 (376, 0.13078335)]

In [29]:
rec_ids = np.array(related)[:, 0]

In [30]:
titles.query("id in @rec_ids")

,id,title
237,356,AllThingsD
245,366,Galerie Daniel Maghen - Planches et dessins or...
250,371,33 Digital Painting Tutorials | 10Steps.SG
251,374,The Gnomon Workshop
253,376,Usability Post
256,380,Alltop - Top User Interface News
258,383,Vectips
263,388,A Detailed Look at HTML5 for UX Designers
268,393,35 Excellent Wireframing Resources | Developer...
273,399,Tips for Coding and Designing Usable Web Forms...
